In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
#x,y,c,
poseName = 'Develope'
Coords = np.load("RootCoordinates/"+poseName+"X.npy")
print(Coords)
print(len(Coords))

[[0.512317 0.282106 0.899708 ... 0.524612 0.742433 0.430888]
 [0.510297 0.287524 0.908392 ... 0.522502 0.748045 0.659226]
 [0.514202 0.2848   0.922879 ... 0.524468 0.745214 0.509468]
 ...
 [0.508067 0.303831 0.849263 ... 0.524451 0.734293 0.39533 ]
 [0.506182 0.306568 0.8451   ... 0.524554 0.739802 0.435512]
 [0.504103 0.320136 0.856297 ... 0.522479 0.756102 0.613614]]
870


In [3]:
#Helper Methods:
import math
def calcDistance(x1,y1,x2,y2):
    distance = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

In [4]:
#Get the Mean Skeleton's coordinates
PoseMeanX = np.load("Stats/"+poseName+"MeanX.npy")
print(PoseMeanX)
print(len(PoseMeanX))
PoseMeanY = np.load("Stats/"+poseName+"MeanY.npy")
print(PoseMeanY)
print(len(PoseMeanY))
MeanTuples = list(zip(PoseMeanX, PoseMeanY))
print(MeanTuples)

[0.50421364 0.50278611 0.47652367 0.4519959  0.45150167 0.52961797
 0.57988041 0.62148406 0.5        0.48722079 0.50148275 0.51339776
 0.51166115 0.46864638 0.41332638 0.4973022  0.51162682 0.47431067
 0.50475292 0.37658326 0.35908833 0.40467099 0.48031957 0.48307012
 0.52011894]
25
[-0.30748475 -0.36691247 -0.36318959 -0.2977106  -0.23294622 -0.3712514
 -0.38846922 -0.39281288 -0.5        -0.49991995 -0.62365885 -0.73810095
 -0.50045429 -0.52171479 -0.53904012 -0.30015452 -0.30007842 -0.30592204
 -0.30582298 -0.53123482 -0.51179463 -0.53731921 -0.7581939  -0.75291908
 -0.751062  ]
25
[(0.5042136425287356, -0.3074847528735634), (0.5027861103448276, -0.3669124655172413), (0.4765236666666664, -0.3631895850574708), (0.45199590344827667, -0.2977106000000003), (0.451501671264368, -0.23294622298850615), (0.5296179655172407, -0.3712513977011492), (0.5798804057471261, -0.38846922068965545), (0.6214840551724147, -0.39281287701149425), (0.5, -0.5), (0.4872207862068965, -0.4999199459770113), (0.5

In [5]:
POSE = "Develope"
allDistanceVectors = []
for sample in Coords:
    xCoords = sample[0::3]
    #print(xCoords)
    yPos = sample[1::3]
    yCoords = [-y for y in yPos] # need to make it negative to be consistent - the mean skeleton's y saved the negative coords.
    SampleTuples = list(zip(xCoords, yCoords))
    #print(SampleTuples)
    longZip = list(zip(MeanTuples, SampleTuples))
    #print(longZip)
    distanceVector = []
    for ((x1,y1),(x2,y2)) in longZip:
        d = calcDistance(x1,y1,x2,y2)
        #print(d)
        distanceVector.append(d)
    allDistanceVectors.append(distanceVector)

print(allDistanceVectors)
print(len(allDistanceVectors))
print(len(allDistanceVectors[0]))
np.save("DataForSVM/"+POSE+"DistanceVectorsPerSample.npy", allDistanceVectors)

[[0.026641049148344375, 0.019353053341240024, 0.018559434600869725, 0.029444237986468905, 0.03615700752397584, 0.0185229231181207, 0.016585266441281176, 0.012314748062804207, 0.0, 0.005458017405857805, 0.011777038498962684, 0.012252347538741902, 0.007810106366728501, 0.09939081920774091, 0.22093128131447357, 0.02405011793236731, 0.02436670748507775, 0.024169581840762417, 0.02685147711659659, 0.2475537670526477, 0.28151217089570946, 0.23576388624022865, 0.010734076244029557, 0.011360720191185801, 0.009728684807665497], [0.02086717262598508, 0.014162716309499536, 0.013444245502160187, 0.026543243969891883, 0.030470339028426222, 0.01293057490692969, 0.011261341322265393, 0.008551914998676911, 0.0, 0.005549134558007895, 0.0016742085422181748, 0.0019517151186512452, 0.0023830795819904456, 0.10700003591018666, 0.22711874165313461, 0.01838754424384617, 0.018607374834735697, 0.021970829421948317, 0.024085146763206187, 0.2490899823727747, 0.28622591679933473, 0.23847138540292354, 0.003617591741

In [6]:
#These features are mixing coordinates and distances: 100 features per sample: 75 + 25
PoseAugFeaturesX = []
for i in range(len(Coords)):
    newList = list(Coords[i])+list(allDistanceVectors[i])
    PoseAugFeaturesX.append(newList)
    
print(PoseAugFeaturesX[0])
print(len(PoseAugFeaturesX[0]))
np.save("DataForSVM/"+POSE+"AugMeanDistFeaturesX.npy", PoseAugFeaturesX)

[0.5123169999999999, 0.28210599999999997, 0.899708, 0.502177, 0.34756899999999996, 0.9382649999999999, 0.475603, 0.344653, 0.837824, 0.455152, 0.268436, 0.871942, 0.46731999999999996, 0.20043299999999997, 0.8408019999999999, 0.53269, 0.352985, 0.870471, 0.5818049999999999, 0.371996, 0.895778, 0.6287689999999999, 0.382884, 0.836816, 0.5, 0.5, 0.732335, 0.485865, 0.494633, 0.695734, 0.5021, 0.611898, 0.7873669999999999, 0.516431, 0.7262299999999999, 0.72631, 0.5042009999999999, 0.5027659999999999, 0.689175, 0.504166, 0.6145419999999999, 0.7245229999999999, 0.516532, 0.7343839999999999, 0.7536029999999999, 0.5021399999999999, 0.276596, 0.9021389999999999, 0.5184, 0.276672, 0.9065909999999999, 0.489926, 0.287474, 0.868124, 0.52664, 0.290268, 0.8007369999999999, 0.485718, 0.7534339999999999, 0.649821, 0.493988, 0.75888, 0.597407, 0.520598, 0.742613, 0.7329669999999999, 0.4838239999999999, 0.748048, 0.478433, 0.487807, 0.742593, 0.442111, 0.5246120000000001, 0.742433, 0.430888, 0.02664104914